<a href="https://colab.research.google.com/github/scottthomaswx/Great-Lakes-WW3/blob/main/GreatLakesWaveWatchIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load and import packages
%%capture
!pip install numpy
!pip install xarray[complete]
!pip install metpy
!pip install matplotlib
!pip install cartopy
!pip install datetime
!pip install requests
!pip install pyproj
!pip install pandas
!pip install cfgrib
!apt-get install -y libeccodes0
!pip install cfgrib xarray
!pip install sys
!pip install warnings
import xarray as xr
import numpy as np
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from datetime import datetime
import requests
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from metpy.plots import add_timestamp
import pyproj as proj
import pandas as pd
import sys
import warnings
import matplotlib.animation as animation
warnings.filterwarnings('ignore')

In [ ]:
#Get and Save WaveWatch III Data
WW = 'https://www.glerl.noaa.gov/emf/waves/WW3/images/nomads.grib2'
r1 = requests.get(WW)
file1='/content/ww.grib2'
with open(file1, 'wb') as f1:
                f1.write(r1.content)

In [ ]:
%%capture
ww3 = xr.open_dataset(file1, engine="cfgrib")
swh = ww3.metpy.parse_cf('swh').metpy.convert_units('ft')

#Map of Wavewatch III Data


In [ ]:
%%capture
!mkdir FcstHourFiles
float_formatter = "{:03d}".format
i=0
rounded = np.array(swh.time, dtype='datetime64[m]')
while i <  len(ww3['valid_time']):
    #Create Figure and Subplot
    fig = plt.figure(figsize=(14,8), facecolor='white', edgecolor = 'black', linewidth = 6)

    ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())

    #Create Wave Contours
    wave_fill = ax.contourf(swh.longitude,swh.latitude,swh.isel(step=i), cmap='viridis_r', levels = np.arange(0,21,1), zorder=7, transform=ccrs.PlateCarree())
    wave_cont = ax.contour(swh.longitude,swh.latitude,swh.isel(step=i), color='black', levels = np.arange(0,21,1), zorder=7, transform=ccrs.PlateCarree())
    wavelabels= ax.clabel(wave_cont, fmt='%01d', fontsize = 'large', colors = 'black')

    #Plot and SUbplot Settings
    plt.colorbar(wave_fill, orientation='vertical',pad=0.02, aspect=50, shrink=0.935, label = 'Wave Height (ft)')
    plt.figtext(0.125, 0.08, f'SCOTT THOMAS | SOURCE: GLERL', weight='bold', color='black', fontsize=16, ha='left', alpha=0.8)
    plt.figtext(0.13, 0.905, f'WW3 Significant Wave Height (Feet)', ha='left', weight='bold', fontsize=15)
    plt.figtext(0.13, 0.875, f'Runtime: {rounded} at FHR: {i}', ha='left', weight='bold', fontsize=12)
    ax.coastlines(color='black',resolution='10m',zorder=99)
    ax.add_feature(cfeature.STATES.with_scale('50m'),edgecolor='black', zorder=9, facecolor = 'grey')

    #Adjust Lat/Long To Map
    ax.set_extent((-88.25,-81,41.5,46.25))
    add_timestamp(ax, swh['valid_time'][i].values.astype('datetime64[ms]').astype('O'),
                  y=0.02, high_contrast=True, color = 'white', pretext = 'Valid: ', zorder=10)
    #Save Figure
    plt.savefig('/content/FcstHourFiles/WW3_FHR'+str(float_formatter(i))+'.png', dpi=100, bbox_inches='tight')
    i+=1